In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

In [2]:
import ipyvuetify as v
import vaex.jupyter.widgets as vw
import ipywidgets as widgets
import vaex
import vaex.jupyter
import numpy as np
import getpass

colors = {
    'primary': '#1976D2',
    'secondary': '#424242',
    'accent': '#82B1FF',
    'error': '#FF5252',
    'info': '#2196F3',
    'success': '#4CAF50',
    'warning': '#FFC107'
}

In [3]:
# Uncomment to access the data via S3
import vaex
import getpass

profile_name = 'stsci' if getpass.getuser() == 'maartenbreddels' else 'default'

df = vaex.open(f's3://astrosurveydata/gaia_ps1_nochunk.hdf5?profile_name={profile_name}')
df = df[:200_000_000]
topic = 'stars'
default_custom_selection = str(df.parallax/df.parallax_error > 3)
default_custom_selection

'((parallax / parallax_error) > 3)'

In [4]:
# import vaex
# with open('token.txt') as f:
#     token = f.read().strip()
# df = vaex.open(f'ws://ec2-18-222-183-211.us-east-2.compute.amazonaws.com:9000/gaia_ps1_nochunk?token_trusted={token}')#[:10_000_000]

In [5]:
df

#,a_g_percentile_lower,a_g_percentile_upper,a_g_val,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_gof_al,astrometric_matched_observations,astrometric_n_bad_obs_al,astrometric_n_good_obs_al,astrometric_n_obs_ac,astrometric_n_obs_al,astrometric_params_solved,astrometric_primary_flag,astrometric_pseudo_colour,astrometric_pseudo_colour_error,astrometric_sigma5d_max,astrometric_weight_al,b,bestdetection,bp_g,bp_rp,dec,dec_error,dec_parallax_corr,dec_pmdec_corr,dec_pmra_corr,designation,dist,duplicated_source,dvoregionid,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,e_bp_min_rp_val,ebv,ecl_lat,ecl_lon,flame_flags,frame_rotator_object_type,g_rp,gapmag,gapmagerr,gdec,gdecerr,gepoch,ginfoflag,ginfoflag2,ginfoflag3,gippdetectid,gkronmag,gkronmagerr,gnframes,gpsfmag,gpsfmagerr,gra,graerr,gstackdetectid,gstackimageid,iapmag,iapmagerr,idec,idecerr,iepoch,iinfoflag,iinfoflag2,iinfoflag3,iippdetectid,ikronmag,ikronmagerr,inframes,ippobjid,ipsfmag,ipsfmagerr,ira,iraerr,istackdetectid,istackimageid,l,lum_percentile_lower,lum_percentile_upper,lum_val,matched_observations,mean_varpi_factor_al,objid,p_dec,p_ra,parallax,parallax_error,parallax_over_error,parallax_pmdec_corr,parallax_pmra_corr,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_bp_n_obs,phot_bp_rp_excess_factor,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_g_n_obs,phot_proc_mode,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_rp_n_obs,phot_variable_flag,pmdec,pmdec_error,pmra,pmra_error,pmra_pmdec_corr,priam_flags,primarydetection,processingversion,projectionid,ra,ra_dec_corr,ra_error,ra_parallax_corr,ra_pmdec_corr,ra_pmra_corr,radial_velocity,radial_velocity_error,radius_percentile_lower,radius_percentile_upper,radius_val,random_index,randomstackobjid,rapmag,rapmagerr,rdec,rdecerr,ref_epoch,repoch,rinfoflag,rinfoflag2,rinfoflag3,rippdetectid,rkronmag,rkronmagerr,rnframes,rpsfmag,rpsfmagerr,rra,rraerr,rstackdetectid,rstackimageid,rv_nb_transits,rv_template_fe_h,rv_template_logg,rv_template_teff,skycellid,solution_id,source_id,surveyid,teff_percentile_lower,teff_percentile_upper,teff_val,tessid,uniquepspsstid,visibility_periods_used,yapmag,yapmagerr,ydec,ydecerr,yepoch,yinfoflag,yinfoflag2,yinfoflag3,yippdetectid,ykronmag,ykronmagerr,ynframes,ypsfmag,ypsfmagerr,yra,yraerr,ystackdetectid,ystackimageid,zapmag,zapmagerr,zdec,zdecerr,zepoch,zinfoflag,zinfoflag2,zinfoflag3,zippdetectid,zkronmag,zkronmagerr,znframes,zpsfmag,zpsfmagerr,zra,zraerr,zstackdetectid,zstackimageid
0,nan,nan,nan,278.35400390625,0.463565430615808,1.49006002867494,2.0987000465393066,27,2,236,0,238,31,False,1.36373796564467,0.0336767775374772,0.2703999876976013,0.5764830112457275,5.3310062088769,1,0.9484689831733704,1.8842600584030151,-10.3091114009876,0.116378626203484,0.04726390168070793,0.29888901114463806,0.12965799868106842,b'Gaia DR2 30410948915',1.18388099435614e-05,False,96139,nan,nan,nan,0.22530299425125122,-31.3804098161502,118.376065081187,-9999,0,0.935791015625,19.6112003326416,0.003068000078201294,-10.3091215,0.0072778500616550446,56075.0212668,102760453,128,196608,7032,19.692699432373047,0.014081999659538269,8,19.599000930786133,0.010356999933719635,114.35555677,0.007036920171231031,2523191306964966264,4379137,17.695100784301758,0.0011759999906644225,-10.30911878,0.001237310003489256,55901.9228214,102760453,128,196608,7032,17.78689956665039,0.004135999828577042,8,412913860963485,17.714000701904297,0.0028639999218285084,114.35555695,0.0013052199501544237,2523204911005440888,4429816,227.395358246246,nan,nan,nan,29,0.00017986699822358787,95621143557289362,-10.30911843,114.35555883,0.957945085087802,0.180076371312301,5.319660186767578,-0.06908410042524338,0.2367289960384369,271.704423086766,6.51448995857181,41.707698822021484,19.26609992980957,24,1.314710021018982,887.719153204297,1.36522767468661,650.2349853515625,18.3176994323

In [6]:

sky_x = df.ra
sky_y = df.dec
limits_sky_x = [0, 360]
limits_sky_y = [-90, 90]

color = df.phot_bp_mean_mag - df.phot_g_mean_mag
df['distance'] = 1/df.parallax
df['M_g'] = df.phot_g_mean_mag - 5 * np.log10(df.distance) - 10
magnitude = df.M_g
limits_color = [-0.5, 1.5]
limit_magnitude = [22, -10]

aux = df.phot_g_mean_mag
limits_aux = [5, 23]
shape_aux = 100

In [7]:
# TODO: vaex cannot work with 'empty' selection, make selection that use all the data
df.select(df.phot_g_mean_mag > 10, name='visual')
df.select(default_custom_selection, name='custom')
df.select(df.ra > -1000, name='sky')
df.select('visual & custom & sky')

# since the default selection depepends on this, and vaex does not
# track the dependencies, we have to update the default selection manually
@df.signal_selection_changed.connect
def update_default_selection(df, selection_name):
    if selection_name in 'visual custom sky'.split():
        df.select('visual & custom & sky')


In [8]:
counter_selection = vw.AnimatedCounter(value=1234567, postfix=f' {topic} selected')
counter_selection.value = df.count(selection=True).item()
counter_selection

AnimatedCounter(characters=['&nbsp;', '&nbsp;', '&nbsp;', '&nbsp;', '3', '4', ',', '7', '7', '7', ',', '0', '1…

In [9]:
counts_aux = df.count(aux, binby=aux, limits=limits_aux, shape=shape_aux, selection=[True, None])
counts_aux

array([[       0,        0,        0,        0,        0,        0,
               0,        0,        0,        0,        0,        0,
               0,        0,        0,        0,        0,        0,
               0,        0,        0,        0,        0,        0,
               0,        0,        0,     3679,    18176,    21051,
           24615,    29183,    33986,    39477,    45367,    52544,
           60383,    70075,    80784,    93063,   106644,   122489,
          140540,   160658,   182886,   212868,   242247,   275035,
          312232,   351910,   396612,   446737,   501199,   559784,
          626743,   694792,   767500,   850557,   938161,  1037426,
         1126665,  1208626,  1294325,  1366736,  1421122,  1454579,
         1459410,  1444085,  1401567,  1341516,  1273030,  1202175,
         1122732,  1049196,   981884,   917166,   852263,   782203,
          689742,   608730,   535436,   461570,   389001,   311729,
          237836,   178923,   109783,    45195, 

In [10]:
df.bin_centers(aux, limits_aux, shape_aux)

array([ 5.09,  5.27,  5.45,  5.63,  5.81,  5.99,  6.17,  6.35,  6.53,
        6.71,  6.89,  7.07,  7.25,  7.43,  7.61,  7.79,  7.97,  8.15,
        8.33,  8.51,  8.69,  8.87,  9.05,  9.23,  9.41,  9.59,  9.77,
        9.95, 10.13, 10.31, 10.49, 10.67, 10.85, 11.03, 11.21, 11.39,
       11.57, 11.75, 11.93, 12.11, 12.29, 12.47, 12.65, 12.83, 13.01,
       13.19, 13.37, 13.55, 13.73, 13.91, 14.09, 14.27, 14.45, 14.63,
       14.81, 14.99, 15.17, 15.35, 15.53, 15.71, 15.89, 16.07, 16.25,
       16.43, 16.61, 16.79, 16.97, 17.15, 17.33, 17.51, 17.69, 17.87,
       18.05, 18.23, 18.41, 18.59, 18.77, 18.95, 19.13, 19.31, 19.49,
       19.67, 19.85, 20.03, 20.21, 20.39, 20.57, 20.75, 20.93, 21.11,
       21.29, 21.47, 21.65, 21.83, 22.01, 22.19, 22.37, 22.55, 22.73,
       22.91])

In [11]:
import bqplot.pyplot as plt
fig1d = plt.figure()
fig1d.fig_margin = {'bottom': 35, 'left': 60, 'right': 5, 'top': 5}
bar = plt.bar(df.bin_centers(aux, limits_aux, shape_aux), counts_aux, type='grouped', colors = [colors['warning'], colors['accent']])
bar.scales['y'].allow_padding = False

fig1d.axes[0].label_offset = '30px'
fig1d.axes[0].label = str(aux)
fig1d.axes[1].label_offset = '50px'
fig1d.axes[1].label = 'counts'

fig1d.axes[0].color = \
fig1d.axes[1].color = \
fig1d.axes[0].label_color = \
fig1d.axes[1].label_color = \
fig1d.axes[0].grid_color = \
fig1d.axes[1].grid_color = '#666'

# even handling
for axes in fig1d.axes:
    axes.grid_lines = 'none'
selector = plt.brush_int_selector()
# plt.show()
fig1d

Figure(axes=[Axis(color='#666', grid_color='#666', grid_lines='none', label='phot_g_mean_mag', label_color='#6…

In [12]:
# listen to the selector, and modify the selections
def update_selection_visual(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        df.select((aux > xmin) & (aux < xmax), name='visual')
selector.observe(update_selection_visual, 'selected')

In [13]:
counter_processed = vw.AnimatedCounter(value=1234567, postfix=f' {topic} processed')
counter_processed.value = len(df)
counter_processed

AnimatedCounter(characters=['&nbsp;', '&nbsp;', '&nbsp;', '2', '0', '0', ',', '0', '0', '0', ',', '0', '0', '0…

In [14]:
progress_circular = vw.ProgressCircularNoAnimation(width=10, size=70, color=colors['accent'], text='')

import time
last_time = time.time()
@df.executor.signal_begin.connect
def progress_begin():
    progress_circular.hidden = False
@df.executor.signal_progress.connect
def update_progress(value):
    global last_time
    progress_circular.value = value*100
    number = int(value * len(df))
    current_time = time.time()
    if (current_time - last_time) > 0.2 or value in [0, 1]:
        counter_processed.value = number
        last_time = current_time
    return True
@df.executor.signal_end.connect
def progress_update():
    progress_circular.hidden = True
    

progress = v.Layout(children=[progress_circular, counter_processed], align_center=True)
progress

Layout(align_center=True, children=[ProgressCircularNoAnimation(color='#82B1FF', size=70, text='', width=10), …

In [15]:
progress_circular.hidden = True

In [16]:
# maybe we should put this in vaex-core?
@vaex.jupyter.debounced(0.05)
def _execute():
    df.execute()

output = widgets.Output()

# this will be called when a selection changes
@df.signal_selection_changed.connect
@vaex.jupyter.debounced(0.5)
def _update_plot1d(df, selection_name):
    if selection_name == 'default':
        with output:
            @vaex.delayed
            def do(grid1d, count):
                with output:
                    bar.y = grid1d
                    counter_selection.value = count.item()
            counts_aux = df.count(aux, binby=aux, limits=limits_aux, shape=shape_aux, selection=[True, None], delay=True)
            do(counts_aux, df.count(selection=True, delay=True))
            _execute()

In [17]:
output

Output()

In [18]:
plot_sky = df.plot_widget(sky_x, sky_y, f='log', selection=[False, 'sky'], limits=[limits_sky_x, limits_sky_y], selection_callback=update_default_selection)
plot_sky.widget.title = f'All {topic}'
c = plot_sky.widget.components['main-widget']
c.layout.padding = '60px 0px 0px 0px'

PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

In [19]:
plot_hrd = df.plot_widget(color, magnitude, f='log', selection=True, limits=[limits_color, limit_magnitude])
plot_hrd.widget.title = f'Selected {topic}'
c = plot_hrd.widget.components['main-widget']
c.layout.padding = '60px 0px 0px 0px'

PlotTemplate(components={'main-widget': VBox(children=(VBox(children=(Figure(axes=[Axis(color='#666', grid_col…

In [20]:
for fig in [fig1d, plot_sky.backend.figure, plot_hrd.backend.figure]:
    fig.layout.width = 'auto'
    fig.layout.width = 'auto'
    fig.layout.height = 'auto'
    fig.layout.min_height = '300px' # so it still shows nicely in the notebook
    fig.layout.min_width = 'auto'
    fig.layout.min_width = 'auto'

In [21]:
expression_area = vw.ExpressionTextArea(df=df)
expression_area

ExpressionTextArea(label='Custom expression', placeholder='Enter a custom expression', prepend_icon='functions…

In [22]:
widget_selection_text_area = vw.ExpressionSelectionTextArea(df=df, selection_name='custom', v_model=default_custom_selection)
widget_selection_text_area

ExpressionSelectionTextArea(label='Filter by custom expression', placeholder='Enter a custom (boolean) express…

In [23]:
widget_selection = vw.SelectionEditor(df=df, input=widget_selection_text_area)
widget_selection

SelectionEditor(components={'component-input': ExpressionSelectionTextArea(label='Filter by custom expression'…

In [24]:
editor = vw.VirtualColumnEditor(df=df)
editor

VirtualColumnEditor(components={'editor': ExpressionTextArea(label='Custom expression', placeholder='Enter a c…

In [25]:
column_list = vw.ColumnList(df=df)
column_list

ColumnList(columns=[{'name': 'a_g_percentile_lower', 'dtype': 'float', 'virtual': False}, {'name': 'a_g_percen…

In [26]:
import traitlets
class Action(v.VuetifyTemplate):
    value = traitlets.Bool(False).tag(sync=True)
    backend = traitlets.Any()
    items = traitlets.Any().tag(sync=True)
    @traitlets.default('template')
    def _template(self):
        return """
        <v-speed-dial
        style="bottom: 80px"
        absolute
      v-model="value"
      bottom
      right
      direction="top"
      transition="slide-y-reverse-transition"
    >
      <template v-slot:activator>
        <v-btn
          v-model="value"
          small
          fab
        >
          <v-icon v-if="value">expand_more</v-icon>
          <v-icon v-else>more_vert</v-icon>
        </v-btn>
      </template>
      <v-tooltip v-for="item in items" left>
          <template v-slot:activator="tooltip">
              <v-btn fab small v-on="tooltip.on" @click="action(item)">
                <v-icon small>{{ item.icon }}</v-icon>
              </v-btn>
          </template>
          {{ item.tooltip }}
      </v-tooltip>
    </v-speed-dial>
    """
    def vue_action(self, data):
        print(data)
        name = data['value']
        self.backend.figure.interaction = self.backend.tool_actions_map[name]
items = [
    {'value': 'pan/zoom', 'icon': 'pan_tool', 'tooltip': "Pan & zoom"},
    {'value': 'select', 'icon':'crop_free', 'tooltip': "Square selection"}, 
]
action = Action(items=items, backend=plot_sky.backend)
v.Html(tag='div', pa_10=True, style_='width: 400px; height: 400px; position: relative', children=[action])

Html(children=[Action(events=['action'], items=[{'value': 'pan/zoom', 'icon': 'pan_tool', 'tooltip': 'Pan & zo…

In [27]:
url1 = "http://www.stsci.edu/modules/stsci-www-assets/assets/img/event-1.jpg?t=tn768"
url2 = "http://www.stsci.edu/modules/stsci-www-assets/assets/img/event-2.jpg?t=tn768"
url1 = url2
gradient="""gradient="to bottom, rgba(255,255,255,.4), rgba(255,255,255,.7)" """
gradient="""gradient="to bottom, rgba(0,0,0,.4), rgba(0,0,0,.7)" """

In [28]:
class MainContent(v.VuetifyTemplate):
    loading = traitlets.Bool(False).tag(sync=True)
    @traitlets.default('template')
    def _template(self):
        return f"""
    <v-container class="grey lighten-5">
        <v-row justify="center">
            <v-col xs="12" lg="6" xl="4">
                <v-card style="height: 100%" :loading="loading"
                    class="mx-auto"
                  >
                    <v-img src="{url1}" class="white--text" height="65px" {gradient}>
                        <v-card-title style="padding-left: 50px;">Overview</v-card-title>
                    </v-img>
                    <v-card-text  style="padding-left: 50px" class="">
                        This <a href="https://github.com/QuantStack/voila">voila</a> dashboard
                        show the <a href="https://gea.esac.esa.int/archive/">Gaia</a> dataset, crossmatched
                        with the <a href="https://panstarrs.stsci.edu/">Pan-STARRS</a> dataset. You are looking at:
                        <p>
                            <ul>
                                <li>{len(df):,} stars.
                                <li><counter/>
                                <li><counter_selection/>
                            </ul>
                        </p>
                        <p>
                            The visualizations are calculated on the fly using <a href="https://github.com/vaexio/vaex/">vaex</a>
                            to process the nearly 1 billion stars interactively.
                        </p>
                        <p>
                            The visualizations on the rights and bottom can be used for filter the data visually.
                            Apart from a visual selection, this following expression is included for filtering as well.
                        </p>
                            <filter-editor/>
                        
                        
                    </v-card-text>
                </v-card>
            </v-col>
            <v-col xs="12" lg="6" xl="4"">
                <v-card  :loading="loading" style="height: 100%" 
                    class="mx-auto"
                  >
                    <v-img src="{url1}" class="white--text" height="65px" {gradient}>
                        <v-card-title style="padding-left: 50px;">Sky view</v-card-title>
                    </v-img>
                    <v-card-text  style="padding-left: 50px">Navigate the sky or select a region of the sky to filter the HR diagram.</v-card-text>
                    <sky/>
                </v-card>
            </v-col>
            <v-col xs="12" lg="6" xl="4">
                <v-card  style="height: 100%"  :loading="loading"
                    class="mx-auto"
                  >
                    <v-img src="{url1}" class="white--text" height="65px" {gradient}>
                        <v-card-title style="padding-left: 50px;">Magnitude distribution</v-card-title>
                    </v-img>
                    <v-card-text  style="padding-left: 50px">Histogram of the Gaia G manitudes. Select a region to filter the HR diagram.</v-card-text>
                    <g-mag/>
                </v-card>
            </v-col>
            <v-col xs="12" lg="6" xl="4">
                <v-card  :loading="loading"
                    class="mx-auto"
                  >
                    <v-img src="{url1}" class="white--text" height="65px"{gradient}>
                        <v-card-title style="padding-left: 50px;">HR diagram</v-card-title>
                    </v-img>
                    <v-card-text  style="padding-left: 50px">
                        Shows a Hertzsprung Russell diagram of the sky selection, the magnitude selection and the custom
                        filter combined.
                    </v-card-text>
                    <hr-diagram/>
                </v-card>
            </v-col>
        </v-row>
    </v-container>
"""
    
widget_main = MainContent(components={
    'sky': v.Html(tag='div', style_='position: relative', children=[plot_sky.backend.widget, action]),
    'hr-diagram': widgets.VBox([plot_hrd.backend.widget]),
    'g-mag': fig1d,
    'counter': counter_processed,
    'counter_selection': counter_selection,
    'filter-editor': widget_selection
})
widget_main

MainContent(components={'sky': Html(children=[VBox(children=(Figure(axes=[Axis(color='#666', grid_color='#666'…

In [29]:
@df.executor.signal_begin.connect
def progress_begin():
    widget_main.loading = True
@df.executor.signal_end.connect
def progress_update():
    widget_main.loading = False


In [30]:
import traitlets
class Main(v.VuetifyTemplate):
    previous = traitlets.List(traitlets.Unicode()).tag(sync=True)
    current = traitlets.List(traitlets.Unicode()).tag(sync=True)
    firsts = traitlets.List(traitlets.Bool(True)).tag(sync=True)
    value = traitlets.Integer()
    format = traitlets.Unicode('{: 14,d}')
    postfix = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('Vaex').tag(sync=True)
    components = traitlets.Dict(None, allow_none=True).tag(sync=True, **widgets.widget.widget_serialization)
    items = traitlets.List().tag(sync=True)
    showNavBar = traitlets.Bool(False).tag(sync=True)
    
    @traitlets.observe('value')
    def _value(self, change):
        text = self.format.format(self.value)
        self.previous = self.current
        self.current = [k.replace(' ', '&nbsp;') for k in text]
        if self.previous is None:
            self.current = self.previous

    template = traitlets.Unicode(f'''
        <v-layout>
            <v-navigation-drawer v-model="showNavBar" absolute app temporary style="width: 400px">
                
                <content-nav/>
            </v-navigation-drawer>
            <v-app-bar app absolute>
                <v-app-bar-nav-icon
                        @click.stop="showNavBar = !showNavBar"
                ></v-app-bar-nav-icon>
                <v-toolbar-title>
                    <h2 style="color: #666">
                    <img style="height: 30px" src="https://vaex.io/img/logos/logo.svg"></img>
                    Gaia Discovery Engine with <a href="https://github.com/vaexio/vaex/">vaex</a> - 1 billion stars
                    </h2>
                </v-toolbar-title>
                <div class="flex-grow-1"></div>
                <progress_circular/>
            </v-app-bar>

            <v-content>
                <content-main/>
            </v-content>
        </v-layout>
      ''').tag(sync=True)

main = Main(items=[{'title': 'Bla'}],
           components={
               'content-main': widget_main,
               'content-nav': column_list,
               'content-progress': progress,
               'content-filter': widget_selection,
               'progress_circular': progress_circular
           })
main

Main(components={'content-main': MainContent(components={'sky': Html(children=[VBox(children=(Figure(axes=[Axi…

In [31]:
v.Layout( _metadata={'mount_id': 'content'}, children=[main]);

In [32]:
progress_circular

ProgressCircularNoAnimation(color='#82B1FF', hidden=True, size=70, text='', width=10)

In [33]:
# plot_hrd.progress.layout.visibility = 'visible'
# plot_sky.progress.layout.visibility = 'visible'

In [34]:
# plot2d_selection.progress.style.bar_color = colors['accent']
# plot2d_selection.progress.style.bar_color = colors['accent']

In [35]:
# plot2d_selection.progress.value = 0.3

In [36]:
# plot2d_selection.backend.figure.title = 'HR diagram'

In [37]:
# plot2d_all.backend.figure.title = ''

In [38]:
# fig1d.title = 'Magnitude cut'

In [39]:
fig1d.background_style = {'fill': 'none'}

In [40]:
plot_hrd.backend.figure.background_style = {'fill': 'none'}
plot_sky.backend.figure.background_style = {'fill': 'none'}

In [41]:
# plot2d_selection.backend.figure.background_style
plot_hrd.backend.figure.axes[1].grid_lines = 'none'

In [42]:
progress_circular.value = 80

In [43]:
progress_circular.size = 30
progress_circular.width = 5

In [44]:
progress_circular.color = '#666'

In [45]:
widget_main.loading = False